In [ ]:
#this file checks different hyperparamenters of LMS and returns the value that generates the highest SNR

In [33]:
import numpy as np
import matplotlib.pylab as plt
import padasip as pa
import torch
from scipy.io import wavfile
import os
import matplotlib.pylab as plt
import padasip as pa
import numpy as np
from scipy.io import wavfile
from scipy.fftpack import fft
from IPython.display import Audio
import IPython

In [34]:
# creation of data
tests = ["Mic_on_Top-2/testing", "Mic_on_Top-2/training"]
commands = [
    "up",
    'go',
    "Noise"
]
labels = []
files = []

In [35]:
#put data into more useful datastructure
for path in tests:
    for file in os.listdir(path):
        command = file.split(".")[0]
        file_name = path+"/"+file
        files.append(file_name)
        labels.append(commands.index(command))

#Suplement with old noise data to make up for different quantities of data
for file in os.listdir("Recordings_Keyword_Noise/Noise/Medium"):
    command = "Noise"
    file_name = "Recordings_Keyword_Noise/Noise/Medium"+"/"+file
    files.append(file_name)
    labels.append(commands.index(command))

for file in os.listdir("Recordings_Keyword_Noise/Noise/Big"):
    command = "Noise"
    file_name = "Recordings_Keyword_Noise/Noise/Big"+"/"+file
    files.append(file_name)
    labels.append(commands.index(command))

recordedData = []
for i in range(len(labels)):
    recordedData.append((files[i], labels[i]))

In [36]:
#Set seeds for consistency
np.random.seed(0)
torch.manual_seed(0)

#shuffle data
np.random.shuffle(recordedData)

def zs(a):
    """ 1d data z-score """
    a -= a.mean()
    return a / a.std()

#snr Calculations
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

FilteredDirectory = "./FilteredData/"



In [40]:
# constants
SAMPLERATE = 44100
n = 300 # filter size
D = 100 # signal delay
i = 0

currentMaxSNR = 0
bestMu = None
#Create filter, it learns as we filter so we want to use the same one
for mu in np.linspace(0.0001, 0.001, num=10):
    f = pa.filters.AdaptiveFilter(model="NLMS", n=n, mu=mu, w="zeros")
    SNRs = []
    for entry in recordedData:
        FILENAME = entry[0]


        # open and process source data
        fs, data = wavfile.read(FILENAME)
        y = data
        y = y.astype("float64")
        y = zs(y) / 10
        N = len(y)

        # contaminated with noise
        d = y 

        # prepare data for simulation
        x = pa.input_from_history(d, n)[:-D]
        d = d[n+D-1:]
        y = y[n+D-1:]

        # apply filter
        yp, e, w = f.run(d, x)
        SNRs.append(signaltonoise(e))
        #Save file
        #saveFile = FilteredDirectory+commands[entry[1]]+"/"+commands[entry[1]]+str(i)+".wav"
        #print(saveFile)
        #wavfile.write(saveFile, 44100, e)
        i+=1
    print(SNRs)
    print("Average SNR: ", sum(SNRs)/len(SNRs))
    if sum(SNRs)/len(SNRs) > currentMaxSNR:
        currentMaxSNR = sum(SNRs)/len(SNRs)
        bestMu = mu

print(bestMu)

[('Recordings_Keyword_Noise/Noise/Big/big50000.3.wav', 2), ('Mic_on_Top-2/training/go.2.wav.4k6m799l.ingestion-6d6cc58874-2ngxd.s2.wav', 1), ('Recordings_Keyword_Noise/Noise/Big/big15000.3.wav', 2), ('Mic_on_Top-2/training/up.6.wav.4k6n055v.ingestion-6d6cc58874-dnqzp.s10.wav', 0), ('Mic_on_Top-2/testing/Noise.up.7.wav.4k6p1chv.ingestion-6d6cc58874-dnqzp.s34.wav', 2), ('Recordings_Keyword_Noise/Noise/Medium/medium50000.1.wav', 2), ('Mic_on_Top-2/testing/go.6.wav.4k6m79au.ingestion-6d6cc58874-dnqzp.s16.wav', 1), ('Mic_on_Top-2/training/up.1.wav.4k6n052g.ingestion-6d6cc58874-2ngxd.s7.wav', 0), ('Mic_on_Top-2/training/up.3.wav.4k6n056v.ingestion-6d6cc58874-dnqzp.s16.wav', 0), ('Mic_on_Top-2/training/up.2.wav.4k6n055u.ingestion-6d6cc58874-kt9b5.s11.wav', 0), ('Mic_on_Top-2/training/go.2.wav.4k6m799l.ingestion-6d6cc58874-2ngxd.s3.wav', 1), ('Mic_on_Top-2/training/up.5.wav.4k6n056u.ingestion-6d6cc58874-kt9b5.s3.wav', 0), ('Recordings_Keyword_Noise/Noise/Medium/medium42500.3.wav', 2), ('Mic_on

/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_55330/1369101386.py:19: WavFileWarning: Reached EOF prematurely; finished at 88244 bytes, expected 88252 bytes from header.
  fs, data = wavfile.read(FILENAME)
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_55330/1369101386.py:19: WavFileWarning: Reached EOF prematurely; finished at 88246 bytes, expected 88254 bytes from header.
  fs, data = wavfile.read(FILENAME)
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_55330/1369101386.py:19: WavFileWarning: Reached EOF prematurely; finished at 66780 bytes, expected 66788 bytes from header.
  fs, data = wavfile.read(FILENAME)
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_55330/1369101386.py:19: WavFileWarning: Reached EOF prematurely; finished at 91242 bytes, expected 91250 bytes from header.
  fs, data = wavfile.read(FILENAME)
/var/folders/m2/gtl2q41d50nd5fp8cknb4jqr0000gp/T/ipykernel_55330/1369101386.py:19: WavFileWarning: Reached EOF prematurely; 

[array(0.00013605), array(-8.20363135e-06), array(-0.00078166), array(-0.00053314), array(0.00127584), array(-6.31729338e-05), array(-0.00056408), array(3.04652946e-05), array(-0.00178621), array(0.0012266), array(7.16477365e-05), array(0.00200178), array(0.00038085), array(-0.00141082), array(-0.00061842), array(0.00128607), array(0.00029542), array(-5.36619001e-05), array(-0.00192172), array(-9.36844409e-05), array(0.00103872), array(8.72347746e-05), array(-0.00089593), array(0.00053602), array(-0.00123863), array(0.00500034), array(0.00059198), array(-7.81060412e-05), array(0.00359594), array(-0.00020943), array(0.0012017), array(-0.00173059), array(0.0007307), array(0.00111259), array(0.00085034), array(-0.00405521), array(-0.00227067), array(-0.0055661), array(0.00730709), array(0.00013534), array(-0.00224645), array(-0.00014914), array(-9.95583061e-05), array(-0.00056829), array(0.00027721), array(0.00214252), array(-0.00074276), array(0.00132211), array(-0.00260047), array(0.000

In [41]:
print(currentMaxSNR)

2.8669193489010753e-05


In [38]:
# display sound players for human validation
print("Original record:")
IPython.display.display(Audio(y, rate=SAMPLERATE))

print("Filtered out noise record:")
IPython.display.display(Audio(yp, rate=SAMPLERATE))

print("Enhanced record")
IPython.display.display(Audio(e, rate=SAMPLERATE))


Original record:


Filtered out noise record:


Enhanced record


In [39]:
print("Original SNR: ", signaltonoise(y))
print("Enhanced SNR: ", signaltonoise(e))

Original SNR:  -3.5932646672894894e-05
Enhanced SNR:  -0.0005995374021677608
